In [156]:
from GlobalLocalTransformer import *
import torch as nn
import torch.nn as nn

import pandas as pd
import os
import scipy.ndimage as nd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error
import numpy as np
import torch.nn.functional as F

In [110]:
# create input embeddings 'zlist'

# 5 represents the number of channels or the depth of the 2D slices
# 130 * 170 represent the height and width of a 2D slice
# torch.rand()

x1 = torch.rand(10,5,128,128)
    
mod = GlobalLocalBrainAge(5,
                    patch_size=32,
                    step=32,
                    nblock=6,
                    backbone='vgg8')
zlist = mod(x1)
for z in zlist:
    print(z.shape)
print('number is:',len(zlist))
# print(zlist)
# print(x1[:,:,0,0])

torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
torch.Size([10, 1])
number is: 10


In [219]:
src = "/media/dataanalyticlab/Drive2/MANSOOR/Dataset/Test/"
X = np.load(src + "VBM_OpenBHB.npy")
Y = pd.read_csv(src + "SubInfoOpenBHB.csv")
print("Data matrix loaded... shape: ", X.shape )
labels = Y.loc[:,"age"]
x = X[:,:,:,60:65,0]
print(type(x))

Data matrix loaded... shape:  (20, 121, 145, 121, 1)
<class 'numpy.ndarray'>


In [220]:
X_train, X_test, Y_train, Y_test = train_test_split(x,labels, test_size=0.2, random_state=42)

X_train = torch.from_numpy(X_train)
X_train = X_train.permute(0,3,1,2)
print(X_train.shape)


X_test = torch.from_numpy(X_test)
X_test = X_test.permute(0,3,1,2)
print(X_test.shape)



Y_train = torch.from_numpy(np.array(Y_train))
Y_test = torch.from_numpy(np.array(Y_test))

print(Y_train.shape, Y_test.shape)


torch.Size([16, 5, 121, 145])
torch.Size([4, 5, 121, 145])
torch.Size([16]) torch.Size([4])


In [229]:
import torch.optim as optim

def train(model, X, labels, batch_size=64, num_epochs=5):
    # set MAE as the loss function for brain age predciction 
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    iters, losses, train_acc, val_acc = [], [], [], []

    # training
    n = 0 # the number of iterations
    for epoch in range(num_epochs):
        for i in range(len(X)):
            zlist = model(X)             # forward pass
            prob = F.softmax(zlist[0][i], dim=0) 
            loss = criterion(prob, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch

            # save the current training information
            iters.append(n)
            losses.append(float(loss)/batch_size)             # compute *average* loss
            train_acc.append(get_accuracy(model, X,labels)) # compute training accuracy 
            print(train_acc)
            # val_acc.append(get_accuracy(model, X_test))  # compute validation accuracy
            n += 1
    return train_acc

In [234]:
def get_accuracy(model, X_set, Y_set):
   
    correct = 0
    total = 0
    for i in range(len(X_set)):
        output = model(X_set) # We don't need to run F.softmax
        pred = output[0].max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(Y_set.view_as(pred)).sum().item()
        total += X_set[i].shape[0]
    return correct / total

In [235]:
model = GlobalLocalBrainAge(5,
                        patch_size=128,
                        step=32,
                        nblock=6,
                        backbone='vgg8')
# 
model = model.double()
acc = train(model, X_train, Y_train)
# zlist = model(x)
# for z in zlist:
    # print(z.shape)


/home/dataanalyticlab/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


[0.0]
[0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0]


In [181]:
# prob = F.softmax(zlist[0], dim=0)
print(zlist[0][5])
# print(sum(prob[0]))

tensor([-0.2627], dtype=torch.float64, grad_fn=<SelectBackward0>)


In [177]:
m = torch.Tensor(zlist[0])
m

tensor([[-0.4166],
        [-0.3347],
        [-0.4152],
        [-0.2939],
        [-0.2174],
        [-0.2627],
        [-0.4650],
        [-0.2199],
        [-0.3386],
        [-0.0836],
        [-0.3961],
        [-0.3762],
        [-0.2319],
        [-0.3487],
        [-0.2062]], dtype=torch.float64, grad_fn=<AliasBackward0>)

In [ ]:
mod = ViTForClassfication( 7, 1)
a = mod(m)

In [163]:
class ViTForClassfication(nn.Module):
    """
    The ViT model for classification.
    """

    def __init__(self, hidden_size, num_classes):
        super().__init__()
        # self.config = config
        # self.image_size = config["image_size"]
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.zlist = zlist
        # Create the embedding module
        # self.embedding = Embeddings(config)
        # Create the transformer encoder module
        # self.encoder = Encoder(config)
        # Create a linear layer to project the encoder's output to the number of classes
        self.classifier = nn.Linear(self.hidden_size, 1)
        # Initialize the weights
        # self.apply(self._init_weights)

    def forward(self, zlist, output_attentions=False):
        # Calculate the embedding output
        # embedding_output = self.embedding(x)
        encoder_output = self.zlist
        # Calculate the encoder's output
        # encoder_output, all_attentions = self.encoder(embedding_output, output_attentions=output_attentions)
        # Calculate the logits, take the [CLS] token's output as features for classification
        logits = self.classifier(encoder_output)
        # Return the logits and the attention probabilities (optional)
        if not output_attentions:
            return (logits, None)
        else:
            return (logits, all_attentions)

In [178]:
# define the loss function
criterion = nn.MSELoss()
# Define the optimizer
#optimizer = torch.optim.Adam(model.parameters(), lr=lr)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
# initialize SaveBestModel class
save_best_model = SaveBestModel(save_dir=save_checkpoint)

# training
def train(model, trainloader, optimizer, criterion):
    model.train()
    print('Training')
    train_running_loss = 0.0
    counter = 0
    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        counter += 1
        inputs, targets = data
        image = inputs.to(device)
        labels = targets.to(device)
        labels = labels.reshape((labels.shape[0], 1))
        labels = labels.float()

        optimizer.zero_grad()
        # forward pass
        outputs = model(image)
        # calculate the loss
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        # backpropagation
        loss.backward()
        # update the optimizer parameters
        optimizer.step()

    # loss for the complete epoch
    epoch_loss = train_running_loss / counter

    return epoch_loss

NameError: name 'lr' is not defined